# PromptTemplat

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os
print(os.environ['LANGSMITH_PROJECT'])

CH02-Prompt


프롬프트 단계는 검색기에서 검색된 문서들을 바탕으로 **언어 모델이 사용할 질문이나 명령을 생성하는 과정** 입니다. 이 단계는 검색된 정보를 바탕으로 최종 사용자의 질문에 가장 잘 대응할 수 있는 응답을 생성하기 위해 필수적인 단계입니다.

## 프롬프트의 필요성

1. 문맥(Context) 설정 : </br>
프롬프트는 언어 모델이 특정 문맥에서 작동하도록 설정하는 역할을 합니다. 이를 통해 모델은 제공된 정보를 바탕으로 보다 정확하고 관련성 높은 답변을 생성할 수 있습니다.

2. 정보 통합 : </br>
여러 문서에서 검색된 정보는 서로 다른 관점이나 내용을 포함할 수 있습니다. 프롬프트 단계에서 이러한 정보를 통합하고, 모델이 이를 효율적으로 활용할 수 있는 형식으로 조정합니다.

3. 응답 품질 향상 : </br>
잘 구성된 프롬프트는 모델이 보다 정확하고 유용한 정보를 제공하게 돕습니다.



### RAG 프로프트 구조

- 지시사항 (Instruction)
- 질문 (사용자 입력 질문)
- 문맥 (검색된 정보)

LLM 객체를 정의한다.

In [3]:
# Load model directly
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B", cache_dir='../models/')
# model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B", cache_dir='../models/')

tokenizer = AutoTokenizer.from_pretrained("beomi/Llama-3-Open-Ko-8B", cache_dir='../models/')
model = AutoModelForCausalLM.from_pretrained("beomi/Llama-3-Open-Ko-8B", cache_dir='../models/')

tokenizer.pad_token = tokenizer.eos_token

def model_generate(prompt):
    device = 'mps' if torch.backends.mps.is_available() else 'cpu'
    encode_token = tokenizer(prompt.text, return_tensors='pt', padding=True, truncation=True)
    model.to(device)

    input_length = encode_token["input_ids"].shape[1]

    with torch.no_grad():
        output = model.generate(
            input_ids = encode_token['input_ids'].to(device),
            attention_mask = encode_token['attention_mask'].to(device),
            temperature = 0.6,
            max_new_tokens = 100
        )

    genearted_ids = output[0][input_length:]
    generated_text = tokenizer.decode(genearted_ids, skip_special_tokens=True)

    return generated_text


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

## 방법 1. `from_template()` 메소드를 사용하여 `ProptTemplate` 객체 생성

- 치환될 변수를 `{변수}` 로 묶어서 템플릿을 정의합니다.

In [4]:
from langchain_core.prompts import PromptTemplate

# template = "Where is {country}'s capital?"
template = "{country}의 수도는 어디입니까?"

prompt_template = PromptTemplate.from_template(template)
prompt_template


PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디입니까?')

In [5]:
# prompt = prompt_template.format(country='South Korea')
prompt = prompt_template.format(country='대한민국')
prompt

'대한민국의 수도는 어디입니까?'

In [6]:
chain = prompt_template | model_generate

In [7]:
chain.invoke({'country' : '대한민국'}) # country 변수에 입력된 값이 자동으로 치환되어 수행됨

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


RuntimeError: MPS backend out of memory (MPS allocated: 27.15 GB, other allocations: 384.00 KB, max allowed: 27.20 GB). Tried to allocate 64.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [3]:
template = "What are the capitals of {country1} and {country2}?"
prompt_template = PromptTemplate.from_template(template)

# 예제 입력
formatted_prompt = prompt_template.invoke({"country1": "South Korea", "country2": "Japan"})
print("Formatted Prompt:", formatted_prompt) 

Formatted Prompt: text='What are the capitals of South Korea and Japan?'
